In [1]:


import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI
import pyodbc
import os

load_dotenv('.env')

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
driver = os.getenv('DRIVER')

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=Yes"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()


In [78]:
query = '''
SELECT 
	e.Descricao
	, ie.Descricao
	, Variavel.Codigo
	, Variavel.Descricao
	, crv.Valor
	, CAST(crv.DataMedicao AS DATE) AS dataMedicao
FROM EngineCalculo.CalculoResultadoVariavel AS crv
INNER JOIN EngineCalculo.Variavel 
	ON Variavel.Id = crv.VariavelId
INNER JOIN EngineCalculo.CalculoResultado AS ExecucaoCalculoResultado 
	ON ExecucaoCalculoResultado.JobId = crv.JobId
INNER JOIN EngineCalculo.CalculoResultadoEquipamento 
	ON CalculoResultadoEquipamento.CalculoId = ExecucaoCalculoResultado.CalculoId 
    AND CalculoResultadoEquipamento.JobId = ExecucaoCalculoResultado.JobId
INNER JOIN Treetech.ViewEngine_UltimoJob
	ON ViewEngine_UltimoJob.EquipamentoId = CalculoResultadoEquipamento.EquipamentoId 
	AND ViewEngine_UltimoJob.JobId = ExecucaoCalculoResultado.JobId

INNER JOIN Equipamento AS e 
	ON e.Id = CalculoResultadoEquipamento.EquipamentoId
	
INNER JOIN EquipamentoInstalacaoEletrica AS eie 
	ON e.Id = eie.EquipamentoId

INNER JOIN InstalacaoEletrica AS ie 
	ON  ie.Id = eie.InstalacaoEletricaId

WHERE 1 = 1
	--e.Id = 132
	AND Variavel.Codigo IN (
						'TFB1_ON_VALOR' -- Temperatura para formação de bolhas enrolamento 1
						, 'TFB2_ON_VALOR' -- Temperatura para formação de bolhas enrolamento 1
						, 'TFB3_ON_VALOR' -- Temperatura para formação de bolhas enrolamento 1
						, 'TA_OFF' -- Teor de Água por ensaio offline
						, 'H2OP1_ON_VALOR' -- VALOR DO TEOR DE H20 NO PAPEL ENROLAMENTO 1
						, 'H2OP2_ON_VALOR' -- VALOR DO TEOR DE H20 NO PAPEL ENROLAMENTO 2
						, 'H2OP3_ON_VALOR' -- VALOR DO TEOR DE H20 NO PAPEL ENROLAMENTO 3
						, 'MONIT_ENV_ISOL1' -- Monitor do Envelhecimento da Isolação - Enrolamento 1
						, 'MONIT_ENV_ISOL2' -- Monitor do Envelhecimento da Isolação - Enrolamento 2
						, 'MONIT_ENV_ISOL3' -- Monitor do Envelhecimento da Isolação - Enrolamento 3

			)
		AND crv.Valor != 0
GROUP BY 
	e.Descricao
	, ie.Descricao
	, Variavel.Codigo
	, Variavel.Descricao
	, crv.Valor
	, CAST(crv.DataMedicao AS DATE)

ORDER BY CAST(crv.DataMedicao AS DATE) DESC
		, e.Descricao DESC
        '''
resultado = cursor.execute(query)
registro = resultado.fetchall()

In [79]:
colunas = [column[0] for column in cursor.description]

In [80]:
colunas

['Descricao', 'Descricao', 'Codigo', 'Descricao', 'Valor', 'dataMedicao']

In [83]:
colunas = ['Descricao', 'Descricao', 'Codigo', 'Descricao', 'Valor', 'dataMedicao']
dados = [dict(zip(colunas, row)) for row in registro]
df = pd.DataFrame(dados)


In [85]:
df.Descricao.unique()

array(['Monitor do Envelhecimento da Isolação - Enrolamento 1',
       'Valor da Temperatura de Formação de Bolhas no Enrolamento 2 por monitoramento online',
       'Monitor do Envelhecimento da Isolação - Enrolamento 2',
       'Valor da Temperatura de Formação de Bolhas no Enrolamento 3 por monitoramento online',
       'Monitor do Envelhecimento da Isolação - Enrolamento 3',
       'Valor da Temperatura de Formação de Bolhas no Enrolamento 1 por monitoramento online',
       'Teor de Água por ensaio offline'], dtype=object)

In [7]:
colunas = ['Descricao', 'CodigoCalculo', 'Descricao', 'UltimaAtualizacaoCalculo', 'descricaoEquipamento', 'descricaoInstalacaoEletrica', 'HealtIndex']
dados = [dict(zip(colunas, row)) for row in registro]
df = pd.DataFrame(dados)

In [8]:
df

,Descricao,CodigoCalculo,UltimaAtualizacaoCalculo,descricaoEquipamento,descricaoInstalacaoEletrica,HealtIndex
0,Dados Cadastrais do Primário da Bucha,BCH_CAD_PRIM,2023-05-30 02:04:59.103,Autotransformador INAT7-01 Fase A,Itacaiúnas,1.0
1,Dados Cadastrais do Secundário da Bucha,BCH_CAD_SEC,2023-05-30 02:04:59.103,Autotransformador INAT7-01 Fase A,Itacaiúnas,1.0
2,Dados Cadastrais do Terciário da Bucha,BCH_CAD_TERC,2023-05-30 02:04:59.103,Autotransformador INAT7-01 Fase A,Itacaiúnas,1.0
3,Capacitância do Primário da Bucha,BCH_CAP_PRIM,2023-05-30 02:04:59.103,Autotransformador INAT7-01 Fase A,Itacaiúnas,1.0
4,Capacitância da Bucha do Reator,BCH_CAP_REATOR,2023-05-30 02:04:59.103,Autotransformador INAT7-01 Fase A,Itacaiúnas,1.0
...,...,...,...,...,...,...
61,Dados Cadastrais do Resfriamento,IE_TR_RESF_CAD,2023-05-30 02:04:59.103,Autotransformador INAT7-01 Fase A,Itacaiúnas,1.0
62,Eventos do Resfriamento,IE_TR_RESF_EV,2023-05-30 02:04:59.103,Autotransformador INAT7-01 Fase A,Itacaiúnas,1.0
63,Dados Cadastrais do Tanque,IE_TR_TQP_CAD,2023-05-30 02:04:59.103,Autotransformador INAT7-01 Fase A,Itacaiúnas,1.0
64,Eventos do Tanque,IE_TR_TQP_EV,2023-05-30 02:04:59.103,Autotransformador INAT7-01 Fase A,Itacaiúnas,1.0


In [70]:
query = '''
SELECT DISTINCT
			gc.Descricao
            , Calculo.Codigo AS CodigoCalculo
			, Calculo.Descricao
			, ExecucaoCalculoResultado.UltimaAtualizacao AS UltimaAtualizacaoCalculo
            , e.Descricao AS descricaoEquipamento
            , ie.Descricao AS descricaoInstalacaoEletrica
			, ExecucaoCalculoResultado.Resultado AS HealtIndex
        FROM [EngineCalculo].[CalculoResultado]  As ExecucaoCalculoResultado
            INNER JOIN [EngineCalculo].[CalculoResultadoEquipamento]
                ON CalculoResultadoEquipamento.CalculoId = ExecucaoCalculoResultado.CalculoId
                AND CalculoResultadoEquipamento.JobId = ExecucaoCalculoResultado.JobId
            INNER JOIN [Treetech].[ViewEngine_UltimoJob]
                ON ViewEngine_UltimoJob.EquipamentoId = CalculoResultadoEquipamento.EquipamentoId
                AND ViewEngine_UltimoJob.JobId = ExecucaoCalculoResultado.JobId
            INNER JOIN [EngineCalculo].[Calculo]
                ON Calculo.Id = ExecucaoCalculoResultado.CalculoId
            INNER JOIN [EngineCalculo].[CalculoResultadoVariavel] As EntradaVariavel
                ON ExecucaoCalculoResultado.JobId = EntradaVariavel.JobId
            INNER JOIN [EngineCalculo].[RevisaoVariavel] As RevisaoVariavel
                ON RevisaoVariavel.VariavelId = EntradaVariavel.VariavelId
                AND RevisaoVariavel.Revisao = EntradaVariavel.Revisao
            INNER JOIN [EngineCalculo].[Variavel]
                ON Variavel.Id = EntradaVariavel.VariavelId
            INNER JOIN Equipamento AS e
                ON e.Id = CalculoResultadoEquipamento.EquipamentoId
            INNER JOIN EquipamentoInstalacaoEletrica AS eie
                ON e.Id = eie.EquipamentoId
            INNER JOIN InstalacaoEletrica AS ie
                ON  ie.Id = eie.InstalacaoEletricaId
            INNER JOIN Familia AS f
                ON f.Id = e.FamiliaId
			INNER JOIN EngineCalculo.CategoriaVariavel AS cv
				ON cv.Id = Variavel.CategoriaVariavelId
			LEFT JOIN EngineCalculo.GrupoCalculo AS gc
				ON gc.Id = Calculo.GrupoCalculoId
        WHERE 1 =1
			AND Calculo.UltimaAtualizacao > '2023-06-24'
			AND f.Nome = 'Autotransformador'
            AND e.CodigoOperacional = '05E3'
            AND gc.Descricao = 'Health Index do Parâmetro'
        ORDER BY ExecucaoCalculoResultado.UltimaAtualizacao DESC
				 , ie.Descricao
                 , e.Descricao  '''
resultado = cursor.execute(query)
registro = resultado.fetchall()


In [74]:
colunas = [column[0] for column in cursor.description]

In [72]:
colunas

['Descricao',
 'CodigoCalculo',
 'Descricao',
 'UltimaAtualizacaoCalculo',
 'descricaoEquipamento',
 'descricaoInstalacaoEletrica',
 'HealtIndex']

In [63]:
colunas = ['JobId', 'VariavelId', 'Revisao', 'Valor', 'DataMedicao', 'UltimaAtualizacao']
dados = [dict(zip(colunas, row)) for row in registro]
df = pd.DataFrame(dados)
df

,JobId,VariavelId,Revisao,Valor
0,247281,26,2023-02-16 12:21:20.640,1
1,247281,26,2023-02-16 12:21:20.643,2
2,247281,26,2023-02-16 12:21:20.643,3
3,247281,26,2023-02-16 12:21:20.643,4
4,247281,26,2023-02-16 12:21:20.643,5
...,...,...,...,...
2744005,265015,180,2023-05-30 14:31:19.160,186
2744006,265015,180,2023-05-30 14:31:19.160,187
2744007,265015,180,2023-05-30 14:31:19.160,188
2744008,265015,180,2023-05-30 14:31:19.160,189


In [46]:
query = '''
      SELECT *
        FROM EngineCalculo.CategoriaVariavel
        '''
resultado = cursor.execute(query)

In [47]:
registro = resultado.fetchall()
# columns=['Id', 'ModeloEquipamentoId', 'FamiliaId', 'Identificador', 'Descricao', 'DataFabricacao', 'NumeroSerie', 'Custo', 'DataOperacao', 'CalculoAutoGarantia', 'GarantiaEstoque', 'PeriodoGarantiaEstoqueId', 'GarantiaOperacao', 'PeriodoGarantiaOperacaoId', 'CoordenadaX', 'CoordenadaY', 'CoordenadaZ', 'InicioGarantia', 'TerminoGarantia', 'Patrimonio1', 'Patrimonio2', 'EquipamentoSigmaId', 'ChaveVinculo', 'DataCriacao', 'UsuarioCriacaoId', 'DataAlteracao', 'UsuarioAlteracaoId', 'IdUc', 'TucId', 'CodigoRcp', 'Reserva', 'MunicipioId', 'UarId', 'ComId', 'A1Id', 'A2Id', 'A3Id', 'A4Id', 'A5Id', 'A6Id', 'LivreA2', 'LivreA3', 'LivreA4', 'LivreA5', 'LivreA6', 'UnidadeA2Id', 'UnidadeA3Id', 'UnidadeA4Id', 'UnidadeA5Id', 'UnidadeA6Id', 'CodigoOperacional', 'FaseEletricaId', 'EquipamentoEspecialId', 'VeiculoId', 'ProprietariaId', 'CodigoErp', 'EstacaoOnsId', 'GrupoConsumoId', 'FuncaoTransmissaoEquipamentoId', 'CodificacaoManual', 'FuncaoTransmissaoId', 'FamiliaOnsId', 'MalhaFamiliaOnsId', 'UltimaAtualizacao', 'FamiliaSamOnsId', 'Zona', 'SistemaCoordenadas']

In [48]:
registro

[(1, 'I2_VAR', 'I2 - Variáveis para Índice de Impacto', '', 1, 1, datetime.datetime(2019, 6, 13, 14, 24, 4, 343000), 1, datetime.datetime(2019, 6, 13, 14, 24, 53, 57000)),
 (2, 'IE_VAR', 'IE - Variáveis Gerais para Índice de Estado', '', 1, 1, datetime.datetime(2019, 6, 13, 14, 24, 26, 400000), 1, datetime.datetime(2019, 6, 13, 14, 24, 26, 467000)),
 (6, 'IE_TR_ATV_TMP', 'IE - Parte Ativa - Temperatura enrolamento', '', 1, 1, datetime.datetime(2019, 6, 26, 12, 21, 11, 713000), 1, datetime.datetime(2019, 6, 26, 12, 41, 23, 987000)),
 (10, 'IE_TR_CAD', 'IE - Parte Ativa - Dados Cadastrais', '', 1, 1, datetime.datetime(2019, 6, 26, 12, 24, 19, 347000), 1, datetime.datetime(2019, 6, 26, 12, 25, 22, 500000)),
 (24, 'IE_TR_ATV_GP', 'IE - Parte Ativa - Envelhecimento do Papel (GP)', '', 1, 1, datetime.datetime(2019, 6, 26, 15, 18, 38, 27000), 1, datetime.datetime(2019, 6, 26, 15, 18, 38, 87000)),
 (25, 'IE_TR_ATV_H2OP', 'IE - Parte Ativa - Teor de água no papel', '', 1, 1, datetime.datetime(2

In [42]:
dados = [dict(zip(columns, row)) for row in registro]
df = pd.DataFrame(dados)
df

,Id,ModeloEquipamentoId,FamiliaId,Identificador,Descricao,DataFabricacao,NumeroSerie,Custo,DataOperacao,CalculoAutoGarantia,...,GrupoConsumoId,FuncaoTransmissaoEquipamentoId,CodificacaoManual,FuncaoTransmissaoId,FamiliaOnsId,MalhaFamiliaOnsId,UltimaAtualizacao,FamiliaSamOnsId,Zona,SistemaCoordenadas
0,1,2,106,None,Autotransformador INAT7-01 Fase A,2007-01-01,1ZBR60560,None,NaT,False,...,1,None,True,None,20.0,None,2023-03-29 12:36:34.987,6,None,2
1,2,2,106,None,Autotransformador INAT7-01 Fase B,2007-01-01,1ZBR60561,None,NaT,False,...,1,None,True,None,20.0,None,2022-12-05 19:53:05.610,6,None,2
2,3,2,106,None,Autotransformador INAT7-01 Fase C,NaT,1ZBR60562,None,NaT,False,...,1,None,True,None,20.0,None,2022-12-05 19:53:25.097,6,None,2
3,4,2,106,None,Autotransformador INAT7-02 Fase A,NaT,1ZBR60564,None,NaT,False,...,1,None,True,None,20.0,None,2022-12-06 09:58:00.277,6,None,2
4,5,2,106,None,Autotransformador INAT7-02 Fase B,NaT,1ZBR60565,None,NaT,False,...,1,None,True,None,20.0,None,2022-12-05 19:54:04.767,6,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,251,1,50,None,"Reator RE 66,6 MVAr - 2 - FVm",NaT,None,None,NaT,False,...,1,None,False,None,NaN,None,2023-03-20 11:19:37.933,5,None,2
250,252,1,50,None,"Reator RE 66,6 MVAr - 2-3 - FR",NaT,None,None,NaT,False,...,1,None,False,None,NaN,None,2023-03-20 11:20:51.947,5,None,2
251,253,1,50,None,"Reator RE 66,6 MVAr - 3 - FBr",NaT,None,None,NaT,False,...,1,None,False,None,NaN,None,2023-03-20 11:22:16.593,5,None,2
252,254,1,50,None,"Reator RE 66,6 MVAr - 3 - FAz",NaT,None,None,NaT,False,...,1,None,False,None,NaN,None,2023-03-20 11:23:46.650,5,None,2
